# First usecase just to make sure everything is working fine

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
from dotenv import load_dotenv

load_dotenv()

endpoint = os.getenv("AZURE_MISTRAL_API_ENDPOINT")
model_name = "mistral-small-2503"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(os.getenv("AZURE_API_KEY")),
    api_version="2024-05-01-preview"
)

response = client.complete(
    messages=[
        SystemMessage(content="You are a helpful assistant."),
        UserMessage(content="I am going to Paris, what should I see?"),
    ],
    max_tokens=2048,
    temperature=0.8,
    top_p=0.1,
    model=model_name
)

print(response.choices[0].message.content)

Paris is a city rich in history, culture, and beauty. Here are some must-see attractions and experiences you shouldn't miss:

1. **Eiffel Tower**: An iconic symbol of Paris, you can either admire it from the ground or take a lift to the top for a breathtaking view of the city.

2. **Louvre Museum**: Home to thousands of works of art, including the Mona Lisa and the Winged Victory. The museum is housed in the Louvre Palace, a former royal residence.

3. **Notre-Dame Cathedral**: Although severely damaged by a fire in 2019, the cathedral remains an important historical and architectural landmark. Restoration work is ongoing.

4. **Montmartre**: Known for its bohemian past, charming streets, and the stunning Sacré-Cœur Basilica at its summit. Don't miss Place du Tertre, where local artists create and sell their work.

5. **Champs-Élysées and Arc de Triomphe**: Stroll down the famous avenue, lined with luxury shops and cafes, and visit the iconic Arc de Triomphe at its end.

6. **Seine Riv

# How to analyze an image

Note the following requirements for image analysis:

- The image must be presented in JPEG, PNG, GIF, or BMP format.
- The file size of the image must be less than 4 megabytes (MB).
- The dimensions of the image must be greater than 50 x 50 pixels.


In [19]:
import os
from azure.ai.vision.imageanalysis import ImageAnalysisClient
from azure.ai.vision.imageanalysis.models import VisualFeatures
from azure.core.credentials import AzureKeyCredential

client = ImageAnalysisClient(
    endpoint=os.getenv("AZURE_CV_ENDPOINT"),
    credential=AzureKeyCredential(os.getenv("AZURE_API_KEY"))
)

result = client.analyze_from_url(
    image_url="https://www.zooplus.fr/magazine/wp-content/uploads/2023/05/corgi-1.jpeg", # Binary data from your image file
    visual_features=[VisualFeatures.TAGS]
)

for tag in result["tagsResult"]["values"]:
    print(tag["name"])

animal
mammal
pet
dog
outdoor
dog breed
grass
snout
corgi
ground
brown
looking
